In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
%matplotlib inline

In [26]:
import sympy
a = sympy.Symbol("a", positive=True)
x = sympy.Symbol("x", positive=True)
t = sympy.Symbol("t", positive=True)

In [27]:
sympy.integrate(t* a * t**(a - 1), (t, 0, 1))



a/(a + 1)

In [58]:
def loglik(y, a):
    loglik = np.log(a) + (a - 1) * np.log(y)
    return loglik


In [59]:
from statsmodels.base.model import GenericLikelihoodModel
class a_model(GenericLikelihoodModel):
    def __init__(self, endog, exog=None, **kwds):
        if exog is None:
            exog = np.zeros_like(endog)
        super(a_model, self).__init__(endog, exog, **kwds)
    def nloglikeobs(self, params):
        a = params[0]
        ll = loglik(self.endog, a)
        return -ll
    def fit(self, start_params=None, maxiter=1000, maxfun=1000, **kwds):
        if start_params is None:
            start_params = 7
        return super(a_model, self).fit(start_params=start_params, 
                                        maxiter=maxiter, maxfun=maxfun, **kwds)
    

In [60]:
y = [0.7, 0.8, 0.9]

In [61]:
model = a_model(y)

In [62]:
results = model.fit()

Optimization terminated successfully.
         Current function value: -0.705080
         Iterations: 17
         Function evaluations: 34


In [63]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               a_model Results                                
==============================================================================
Dep. Variable:                      y   Log-Likelihood:                 2.1152
Model:                        a_model   AIC:                            -4.230
Method:            Maximum Likelihood   BIC:                            -4.230
Date:                Mon, 06 Mar 2017                                         
Time:                        08:40:35                                         
No. Observations:                   3                                         
Df Residuals:                       3                                         
Df Model:                          -1                                         
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          4.3784      2.528      1.732      0.083        -0.576     9.333
==============================================================================
"""

In [64]:
from statsmodels.sandbox.regression.gmm import GMM

In [93]:
class GMMGamma(GMM):

    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        # TODO: clean up signature
        kwds.setdefault('k_moms', 2)
        kwds.setdefault('k_params', 2)
        super(GMMGamma, self).__init__(*args, **kwds)


    def momcond(self, params):
        p0, p1 = params
        endog = self.endog
        error1 = endog - p0 / p1
        error2 = endog**2 - (p0 + 1) * p0 / p1**2
        g = np.column_stack((error1, error2))
        return g
y = np.array([0.7, 0.8, 0.9, 0.3, 0.5])
nobs = y.shape[0]
x = np.ones((nobs, ))

model = GMMGamma(y, x, None)
beta0 = np.array([2, 0.1])
res = model.fit(beta0, maxiter=2, optim_method='nm', wargs=dict(centered=False))

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 91
         Function evaluations: 179
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 27
         Function evaluations: 55


In [94]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               GMMGamma Results                               
==============================================================================
Dep. Variable:                      y   Hansen J:                    2.011e-12
Model:                       GMMGamma   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:                Mon, 06 Mar 2017                                         
Time:                        10:30:13                                         
No. Observations:                   5                                         
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          8.8276      5.237      1.686      0.092        -1.437    19.092
x1            13.7931      6.611      2.086      0.037         0.836    26.750
==============================================================================
"""

In [122]:
class GMMGamma(GMM):

    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        # TODO: clean up signature
        kwds.setdefault('k_moms', 2)
        kwds.setdefault('k_params', 1)
        super(GMMGamma, self).__init__(*args, **kwds)


    def momcond(self, params):
        p0, p1 = params
        endog = self.endog
        error1 = endog - p0 / (p0 + 1)
        error2 = endog**2 - p1 
        g = np.column_stack((error1, error2))
        return g
y = np.array([0.7, 0.8, 0.9, 0.3, 0.5])
nobs = y.shape[0]
x = np.ones((nobs, 4))

model = GMMGamma(y, x, None)
beta0 = np.array([2, 0.1])
res = model.fit(beta0, maxiter=2, optim_method='nm')
res.summary()

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 47
         Function evaluations: 89
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 27
         Function evaluations: 51


<class 'statsmodels.iolib.summary.Summary'>
"""
                               GMMGamma Results                               
==============================================================================
Dep. Variable:                      y   Hansen J:                    1.390e-08
Model:                       GMMGamma   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:                Mon, 06 Mar 2017                                         
Time:                        10:39:24                                         
No. Observations:                   5                                         
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x2             1.7777      0.743      2.392      0.017         0.321     3.234
x3             0.4560      0.116      3.929      0.000         0.229     0.683
==============================================================================
"""